In [ ]:
# N_bkg는 background MC weight 합 (위에서 계산한 bkg_w)
N_asimov = bkg_w   # 관측치 대신 Asimov 데이터


In [ ]:
import pyhf

# signal_data: [N_sig]      (mass 포인트별로 채워 줄 값)
# bkg_data:    [N_bkg]
# bkg_uncerts: [0.0]         (stat-only)
model = pyhf.simplemodels.hepdata_like(
    signal_data=[N_sig],
    bkg_data=[bkg_w],
    bkg_uncerts=[0.0],
)

# Asimov 관측치
data_asimov = [N_asimov]


In [ ]:
results = pyhf.infer.intervals.upper_limits(
    model,
    data_asimov,
    q_profile=False,
    return_expected=True
)

# 반환된 dict 예시 키: 'observed', 'expected'
print(f"Expected 95% UL = {results['expected']:.2f}")


In [ ]:
import matplotlib.pyplot as plt

masses     = [100, 200, 300, 400, 500]  # 예시
exp_limits = []

for m in masses:
    # --- signal yield 재계산 ---
    sig_file = uproot.open(f"sig_mass{m}.root")["Events"]
    mask_sig = make_mask(sig_file)
    N_sig    = sig_file["weight"].array()[mask_sig].sum()

    # --- 모델 & Asimov 데이터 업데이트 ---
    model = pyhf.simplemodels.hepdata_like([N_sig], [bkg_w], [0.0])
    data_asimov = [bkg_w]

    # --- limit 계산 ---
    res = pyhf.infer.intervals.upper_limits(model, data_asimov,
                                            q_profile=False,
                                            return_expected=True)
    exp_limits.append(res["expected"])

# --- Plot ---
plt.figure()
plt.step(masses, exp_limits, where='mid', linestyle='--',
         label='Expected (Asimov)')
plt.xlabel('Signal mass [GeV]')
plt.ylabel('95% CL upper limit')
plt.yscale('log')
plt.legend()
plt.title('Stat-only Expected Limit')
plt.show()
